# Step 1: Loading and Initial Data Exploration


In [ ]:
import pandas as pd

prices_panel= pd.read_csv(r"C:\Users\Siwar\Research_project\First approach\Datasets\1_Raw_data_input\prices_data_all_numerical.csv")

prices_panel 

In [ ]:
factors= pd.read_csv(r"C:\Users\Siwar\Research_project\First approach\Datasets\1_Raw_data_input\Weekly Features.csv", index_col=0, parse_dates=['Date'])
factors.index= pd.to_datetime(factors['Date'] , format='%Y-%m-%d')

factors = factors[factors['Date'].isin(prices_panel['Date'])]

factors

## create the season feature

In [ ]:
def assign_season(date):
    if date.month in [12, 1, 2]:
        return 'Winter'
    elif date.month in [3, 4, 5]:
        return 'Spring'
    elif date.month in [6, 7, 8]:
        return 'Summer'
    elif date.month in [9, 10, 11]:
        return 'Autumn'

factors['Season'] = factors.index.to_series().apply(assign_season)
factors

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

factors['Season']= label_encoder.fit_transform(factors['Season'])

season_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("season encoding:", season_mapping) 

# Missing values handling

In [ ]:

missing_values = prices_panel.isnull().sum()

print("Missing Values in Each Column:")
print(missing_values)

prices_panel = prices_panel.apply(lambda col: col.fillna(col.mode()[0]) if col.isnull().any() else col)

missing_values_after_imputation = prices_panel.isnull().sum()

print("\nMissing Values After Imputation:")
print(missing_values_after_imputation)


## join prices dataframe with factors dataframe

In [ ]:
factors['Date']= pd.to_datetime(factors['Date'] , format='%Y-%m-%d')
prices_panel['Date']= pd.to_datetime(prices_panel['Date'] , format='%Y-%m-%d')

prices_panel= prices_panel.reset_index(drop=True)
factors= factors.reset_index(drop=True)

merged_df = pd.merge(prices_panel, factors, on='Date', how='left')

merged_df.index = merged_df['Date']

merged_df = merged_df.drop(columns=['Date'])

merged_df = merged_df.apply(pd.to_numeric, errors='coerce')
merged_df = merged_df.sort_values(by='Series')
merged_df = merged_df.groupby('Series').apply(lambda x: x.sort_index())
merged_df = merged_df.reset_index(level='Series', drop=True)

merged_df= merged_df.dropna()
merged_df

## Split data


In [ ]:
merged_dff = merged_df.sort_index()
merged_dff.to_csv(r"C:\Users\Siwar\Research_project\First approach\Datasets\1_Raw_data_input\merged_df.csv")

In [ ]:

train_size = int(0.8 * len(merged_dff))
merged_dff.index = pd.to_datetime(merged_dff.index)

# Split into train and test sets
train = merged_dff.iloc[:train_size]
test = merged_dff.iloc[train_size:]

# Save train and test sets to CSV
train.to_csv(f'C:\\Users\\Siwar\\Research_project\\Second approach\\Datasets\\2_Train_test_second_approach\\train.csv')
test.to_csv(f'C:\\Users\\Siwar\\Research_project\\Second approach\\Datasets\\2_Train_test_second_approach\\test.csv')

print(f"Train size = {len(train)}, Test size = {len(test)}")


# feature engineering

## price trend

In [ ]:
import pandas as pd

def calculate_price_lag_diff(train_df, test_df):
    """
    Calculate the difference between the current price (t) and the price at lag (t-i) for each series.
    Avoids introducing NaNs in the test set by using historical data from the train set.

    Parameters:
        train_df (pd.DataFrame): Training DataFrame containing 'Series' and 'Price' columns.
        test_df (pd.DataFrame): Testing DataFrame containing 'Series' and 'Price' columns.

    Returns:
        pd.DataFrame, pd.DataFrame: Updated train and test DataFrames with lag difference columns.
    """
    # Combine train and test data temporarily to avoid nan values in the test
    combined_df = pd.concat([train_df, test_df], axis=0)

    # Group by Series
    grouped = combined_df.groupby('Series')

    # Differences for lags: 1 month, 3 months, 6 months, 1 year, 1.5 years
    for i in [4, 12, 24, 48, 72, 96]:  
        combined_df[f'price_lag_diff{i}'] = grouped['Price'].transform(lambda x: x - x.shift(i))

    # Split back into train and test
    train_df = combined_df.iloc[:len(train_df)]
    test_df = combined_df.iloc[len(train_df):]

    return train_df, test_df

def create_price_trend_sma(train_df, test_df):
    """
    Creates price trend simple moving average (SMA) features for various window sizes, grouped by series.
    Avoids introducing NaNs in the test set by using historical data from the train set.

    Parameters:
        train_df (pd.DataFrame): Training DataFrame containing 'Price' and 'Series' columns.
        test_df (pd.DataFrame): Testing DataFrame containing 'Price' and 'Series' columns.

    Returns:
        pd.DataFrame, pd.DataFrame: Updated train and test DataFrames with new price trend SMA features.
    """
    # Combine train and test data temporarily to avoid nan values in the test
    combined_df = pd.concat([train_df, test_df], axis=0)

    # Calculate SMA for various window sizes grouped by Series
    grouped = combined_df.groupby('Series')

    for i in [4, 12, 24, 48, 72, 96]:
        combined_df[f'price_trend_sma{i}'] = grouped['Price'].transform(lambda x: x.rolling(window=i).mean())

    # Split back into train and test
    train_df = combined_df.iloc[:len(train_df)]
    test_df = combined_df.iloc[len(train_df):]

    return train_df, test_df



## Economic factors feature engineering

In [ ]:
!pip install fastdtw pandas matplotlib


In [ ]:
!pip install pyinform

In [ ]:

import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests

def create_lagged_features(df, feature, max_lag):
    for lag in range(1, max_lag + 1):
        df[f'{feature}_lag{lag}'] = df[feature].shift(lag)
    lagged_features = [f'{feature}_lag{lag}' for lag in range(1, max_lag + 1)]
    return df, lagged_features


def perform_granger_causality(df, target, feature, max_lag):
    """
    Performs Granger causality test on a given feature to find the best lag (greater than 4 if possible).

    Parameters:
        df (pd.DataFrame): The dataframe containing time series data.
        target (str): The target column name.
        feature (str): The feature column name.
        max_lag (int): Maximum lag to test.

    Returns:
        best_lag (int): Optimal lag with the most significant p-value (>4 if possible).
        p_values (list): List of p-values for each lag.
    """
    best_lag = 0
    best_p_value = float('inf')
    p_values = []
    lag_p_values = {}

    for lag in range(1, max_lag + 1):
        test_data = df[[target, f"{feature}_lag{lag}"]].dropna()
        if test_data.shape[0] > 1:
            result = grangercausalitytests(test_data, maxlag=lag, verbose=False)
            p_value = result[lag][0]['ssr_chi2test'][1]
            p_values.append(p_value)
            lag_p_values[lag] = p_value

            if p_value < best_p_value:
                best_lag, best_p_value = lag, p_value

    # If best lag is ≤ 4, find the best lag greater than 4
    if best_lag <= 4:
        best_lag_over_4 = min(
            (lag for lag in lag_p_values if lag > 4), 
            key=lambda lag: lag_p_values[lag], 
            default=best_lag
        )
        best_lag = best_lag_over_4

    return best_lag, p_values


def compute_optimal_lags(df, target, features, max_lag, path):
    import pandas as pd
    import statsmodels.api as sm
    import json
    optimal_lags = {}
    p_values_dict = {}

    for feature in features:
        df_copy, _ = create_lagged_features(df.copy(), feature, max_lag)
        optimal_lag, p_values = perform_granger_causality(df_copy, target, feature, max_lag)
        optimal_lags[feature] = optimal_lag
        p_values_dict[feature] = p_values

    with open(path, 'w') as file:
        json.dump(optimal_lags, file)

    return optimal_lags, p_values_dict

def plot_lag_p_values(features, p_values_dict, optimal_lags, max_lag):
    for feature in features:
        p_values = p_values_dict.get(feature, [])
        optimal_lag = optimal_lags.get(feature, 0)

        plt.figure(figsize=(10, 6))
        plt.plot(range(1, len(p_values) + 1), p_values, marker='o', label='P-Values')
        plt.axvline(optimal_lag, color='r', linestyle='--', label=f'Optimal Lag: {optimal_lag}')
        plt.title(f"P-Values for Lags of {feature}")
        plt.xlabel("Lag")
        plt.ylabel("P-Value")

        xticks = range(1, max_lag + 1, 5)
        plt.xticks(xticks)

        plt.axhline(y=0.05, color='g', linestyle='--', label='Significance Level (0.05)')
        plt.legend()
        plt.grid()
        plt.show()

def apply_optimal_lags(df, features, optimal_lags):
    for feature in features:
        optimal_lag = optimal_lags.get(feature, 0)
        if optimal_lag > 0:
            lagged_feature = f'{feature}_lag{optimal_lag}'
            df[lagged_feature] = df[feature].shift(optimal_lag)
            df.drop(columns=[feature], inplace=True)
    return df

In [ ]:
import os
import json
import pandas as pd

def apply_lags(train_df, test_df, json_folder):
    """
    Apply optimal lags to factors already present in the dataset for both train and test data.
    
    Parameters:
        train_df (pd.DataFrame): The training dataset containing series and factor columns.
        test_df (pd.DataFrame): The testing dataset containing series and factor columns.
        json_folder (str): Path to the folder containing JSON files with lags.
    
    Returns:
        pd.DataFrame, pd.DataFrame: The updated train and test datasets with lagged factors.
    """
    # Combine train and test data temporarily to apply lags
    combined_df = pd.concat([train_df, test_df], axis=0)
    
    # Load JSON files
    json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]
    
    for json_file in json_files:
        series_num = int(json_file.split('_')[-1].split('.')[0])

        # Load lags for the current series
        with open(os.path.join(json_folder, json_file), 'r') as f:
            optimal_lags = json.load(f)
        
        # Filter only rows belonging to this series
        mask = combined_df['Series'] == series_num
        series_df = combined_df.loc[mask].copy()  # Copy only the relevant subset

        for factor, lag in optimal_lags.items():
            if factor in series_df.columns:
                series_df[f'{factor}_lagged'] = series_df[factor].shift(lag)

        # Update only the affected rows without modifying others
        combined_df.loc[mask, [col for col in series_df.columns if '_lagged' in col]] = \
            series_df[[col for col in series_df.columns if '_lagged' in col]]

    # Split the combined dataframe back into train and test
    train_with_lags = combined_df.iloc[:len(train_df)]
    test_with_lags = combined_df.iloc[len(train_df):]
    
    return train_with_lags, test_with_lags




## feature transformation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

def find_best_transformation_for_all_features(data, target, threshold=0.1):
    """
    Finds the best transformation for each feature based on R² improvement.
    
    Parameters:
        data (pd.DataFrame): The dataset containing features and target.
        target (str): The name of the target variable.
        threshold (float): Minimum R² improvement required to select a transformation.
    
    Returns:
        dict: A dictionary where keys are feature names and values are their best transformations.
    """
    transformations = {
        'original': lambda x: x,
        'inverse': lambda x: 1 / (x + 1e-10),  
        'squared': lambda x: x**2,
        'cubed': lambda x: x**3
    }
    
    X = data.drop(columns=[target])
    y = data[target]
    
    best_transformations = {}

    for feature in X.columns:
        feature_data = X[[feature]].values
        
        # Compute R² for the original feature
        model = LinearRegression()
        model.fit(feature_data, y)
        base_r2 = r2_score(y, model.predict(feature_data))
        
        best_r2 = base_r2
        best_transformation = 'original'
        
        # Test transformations
        for name, func in transformations.items():
            if name == 'original':
                continue  
            
            try:
                transformed_X = func(feature_data)
                model.fit(transformed_X, y)
                r2 = r2_score(y, model.predict(transformed_X))
                
                if r2 > best_r2 and (r2 - base_r2) >= threshold:
                    best_r2 = r2
                    best_transformation = name
            except Exception as e:
       
                continue
        
        best_transformations[feature] = best_transformation
    
    return best_transformations


def apply_transformations_to_all_features(data, best_transformations):
    """
    Applies the best transformations to all features in the dataset.
    
    Parameters:
        data (pd.DataFrame): The dataset containing features to transform.
        best_transformations (dict): A dictionary where keys are feature names and values are their best transformations.
    
    Returns:
        pd.DataFrame: A new DataFrame with the transformed features.
    """
    transform_functions = {
        'original': lambda x: x,
        'inverse': lambda x: 1 / (x + 1e-10),  
        'squared': lambda x: x**2,
        'cubed': lambda x: x**3
    }
    
    transformed_data = data.copy()
    
    for feature, transformation in best_transformations.items():
        if transformation in transform_functions:
            transformed_data[feature] = transform_functions[transformation](transformed_data[feature])
    
    return transformed_data



## Feature scaling

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def scale_features(train, test, continuous_features, scaler_type='minmax'):
    """
    Scales continuous features and the target in train and test sets using the specified scaler.

    Parameters:
        train (pd.DataFrame): Training dataset.
        test (pd.DataFrame): Test dataset.
        continuous_features (list): List of continuous feature column names.
        target (str): Target column name.
        scaler_type (str): Type of scaler to use ('standard' or 'minmax').

    Returns:
        train_scaled (pd.DataFrame): Scaled training dataset.
        test_scaled (pd.DataFrame): Scaled test dataset.
        target_scaler (scaler object): Fitted scaler object for inverse scaling of target.
    """
    if scaler_type == 'standard':
        scaler = StandardScaler()
  
    elif scaler_type == 'minmax':
        scaler = MinMaxScaler()
   
    else:
        raise ValueError("scaler_type must be 'standard' or 'minmax'")

    # Copy data to avoid modifying original datasets
    train_scaled = train.copy()
    test_scaled = test.copy()

    # Scale continuous features
    train_scaled[continuous_features] = scaler.fit_transform(train[continuous_features])
    test_scaled[continuous_features] = scaler.transform(test[continuous_features])


    return train_scaled, test_scaled



## remove nan

In [ ]:

def align_series_time_range(df, series_col='Series'):
    """
    Aligns all series to the same time range based on the series with the smallest time range.
    Removes NaN values.

    Parameters:
        df (pd.DataFrame): Input dataframe with a DateTime index and multiple series.
        series_col (str): Name of the series column.

    Returns:
        pd.DataFrame: Time-aligned dataframe with NaNs removed.
    """

    # Drop NaNs first
    df = df.dropna()

    # Ensure index is datetime
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("Index must be a DatetimeIndex.")

    # Find the time range for each series
    min_times = df.groupby(series_col).apply(lambda x: x.index.min()).min()
    max_times = df.groupby(series_col).apply(lambda x: x.index.max()).max()

    # Align all series to this range
    df = df.loc[min_times:max_times]

    return df



# feature selection - general

In [ ]:

from sklearn.feature_selection import VarianceThreshold, mutual_info_regression

def feature_selection_with_mutual_info(
    df, target, variance_threshold=0.005, mi_threshold=0.3, max_features=60
):
    """
    Perform feature selection using variance filtering and mutual information.

    Parameters:
        df (pd.DataFrame): DataFrame containing features and target.
        target (str): Name of the target column.
        variance_threshold (float): Threshold for variance-based feature removal.
        mi_threshold (float): Minimum mutual information value to select features.
        max_features (int): Maximum number of features to retain.

    Returns:
        list: Final selected features.
        pd.DataFrame: DataFrame with features, their mutual information scores.
    """
    def remove_low_variance_features(data, threshold):
        """Remove features with variance below the threshold."""
        selector = VarianceThreshold(threshold)
        selector.fit(data)
        retained_columns = data.columns[selector.get_support()]
        return data[retained_columns]
    
    def calculate_mutual_information(X, y):
        """Calculate mutual information between features and target."""
        mi_scores = mutual_info_regression(X, y, random_state=42)
        return pd.DataFrame({
            'feature': X.columns,
            'mutual_information': mi_scores
        }).sort_values(by='mutual_information', ascending=False)
    
    # Step 1: Separate features and target
    X = df.drop(columns=[target])
    y = df[target]
    
    # Step 2: Remove low-variance features
    X_filtered = remove_low_variance_features(X, variance_threshold)
    
    # Step 3: Calculate mutual information scores
    mi_df = calculate_mutual_information(X_filtered, y)
    
    # Step 4: Filter features by mutual information threshold
    selected_features = mi_df[mi_df['mutual_information'] >= mi_threshold]['feature']
    
    # Step 5: Limit to the top `max_features` 
    top_features = selected_features.head(max_features).tolist()

    results_df = mi_df[mi_df['feature'].isin(top_features)]
    
    return top_features, results_df


## MAIN

In [ ]:
import warnings
# Suppress all warnings
warnings.filterwarnings('ignore')


columns_to_exclude = ['month', 'Market', 'Series', 'Width', 'Height', 'Length', 'Grade.Type', 'Season', 'Date', 'y']
continuous_features = [col for col in train.columns.tolist() if col not in columns_to_exclude]

train , test = create_price_trend_sma(train, test)
train , test = calculate_price_lag_diff(train, test)
#train , test = scale_features(train, test, continuous_features, scaler_type='minmax')

# Iterating over Series to compute optimal lags 
for i in range(1, 129):  
    # Filter the DataFrame for the current Series
    df = train[train['Series'] == i].copy()
    df = df.sort_index()


    # List of economic factors (((==>excluding 'Season' and 'Date')))
    economic_factors = factors.columns.tolist()
    economic_factors.remove('Season')
    economic_factors.remove('Date')

    max_lag = 96  # Maximum lag to evaluate
    output_json = f"C:\\Users\\Siwar\\Research_project\\Second approach\\Datasets\\3_Data_ready_for_modelling\\optimal_lags\\optimal_lags_{i}.json"
    target_col = 'y'
    print(f"Processing Product S{int(i)}...")

    # Compute optimal lags 
    optimal_lags, p_values_dict = compute_optimal_lags(df, 'y', economic_factors, max_lag, output_json)

    plot_lag_p_values(economic_factors, p_values_dict, optimal_lags, max_lag)

# Apply optimal lags 
train, test = apply_lags(train, test, json_folder="C:\\Users\\Siwar\\Research_project\\Second approach\\Datasets\\3_Data_ready_for_modelling\\optimal_lags\\")
factors_col = factors.columns.tolist()
factors_col.remove('Date')
factors_col.remove('Season')
test= test.drop(columns=factors_col)
train = train.drop(columns=factors_col)



train = align_series_time_range(train, series_col='Series')
test = align_series_time_range(test, series_col='Series')  
best_transformations = find_best_transformation_for_all_features(train, 'y', threshold=0.1)
train = apply_transformations_to_all_features(train, best_transformations)
test = apply_transformations_to_all_features(test, best_transformations)

selected_features, results = feature_selection_with_mutual_info(
    df=train, 
    target="y", 
    variance_threshold=0.005, 
    mi_threshold=0.4, 
    max_features=60)  
selected_features= selected_features + ['y'] 
train = train[selected_features]
test = test[selected_features]
train.to_csv(r"C:\Users\Siwar\Research_project\Second approach\Datasets\3_Data_ready_for_modelling\train.csv")
test.to_csv(r"C:\Users\Siwar\Research_project\Second approach\Datasets\3_Data_ready_for_modelling\test.csv")